In [1]:
print('hlw world')

hlw world


In [2]:
print('hlw')

hlw


In [3]:
import re
import numpy as np
import pandas as pd
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures
from bnltk.stemmer import BanglaStemmer
from bnltk.tokenize import Tokenizers
from nltk.tokenize import RegexpTokenizer

In [4]:
bangla_stopwords = [
    'অতএব', 'অথচ', 'অথবা', 'অনুযায়ী', 'অনেক', 'অনেকে', 'অন্তত', 'অন্য', 'অবধি', 'অবশ্য', 'অর্থাৎ',
    'আই', 'আছে', 'আজ', 'আপনার', 'আপনি', 'আবার', 'আমরা', 'আমাকে', 'আমাদের', 'আমার', 'আমি',
    'আর', 'আরও', 'ই', 'ইত্যাদি', 'এই', 'এ', 'এঁদের', 'এঁরা', 'এটি', 'এটা', 'এটা', 'এদিকে', 'এবং', 'এবার',
    'এমন', 'এমনকী', 'এরা', 'এল', 'এস', 'এসে', 'ঐ', 'ও', 'ওই', 'ওদের', 'ওর', 'কখনও', 'কত', 'কবে',
    'কমনে', 'কয়েক', 'করবে', 'করবেন', 'করল', 'করলেন', 'করা', 'করানো', 'করায়', 'করার', 'করি', 'করেছে',
    'করেন', 'করে', 'করেই', 'করেনি', 'কিছু', 'কিছুই', 'কিন্তু', 'কী', 'কেউ', 'কে', 'কেমন', 'কোথা', 'কোন',
    'কোনও', 'কি', 'কিভাবে', 'কিছুক্ষণ', 'কেন', 'কেউই', 'গিয়ে', 'গিয়েছে', 'গেল', 'গেলে', 'চলে', 'ছাড়া',
    'ছিল', 'ছিলাম', 'ছিলেন', 'ছিলো', 'জানেন', 'জন্য', 'জানে', 'জানানো', 'জানায়', 'জানিয়ে', 'টা',
    'টি', 'তখন', 'তত', 'তবে', 'তুমি', 'তুলে', 'তেমন', 'তো', 'তোমার', 'তাদের', 'তাহলে', 'তা', 'তাঁদের',
    'তাঁরা', 'তাঁর', 'তাঁরা', 'তাঁকে', 'তাঁরাও', 'তাদেরকে', 'তারা', 'তার', 'তারা', 'তাকে', 'তাহা', 'তিন',
    'তিনি', 'তিনিও', 'তুমি', 'তোমরা', 'থাকবে', 'থাকবেন', 'থাকা', 'থাকায়', 'থাকে', 'থেকেও', 'থেকে',
    'দিকে', 'দিয়ে', 'দিলেন', 'দিতে', 'দিয়ে', 'দুজন', 'দুটি', 'দুটো', 'দেওয়া', 'দেওয়া হয়', 'দেন', 'দেননি',
    'ধরে', 'ধরে', 'নই', 'নয়', 'না', 'নিতে', 'নিজে', 'নিজেই', 'নিজের', 'নিজেদের', 'নেওয়া', 'নেওয়া হয়নি',
    'নেওয়ার', 'নিয়েই', 'নিয়েছে', 'নিয়েছেন', 'নিয়ে', 'নেই', 'পক্ষে', 'পর', 'পরেও', 'পর্যন্ত', 'পরে', 'পাওয়া',
    'পারে', 'পারেন', 'পারি', 'পি', 'পেয়েছি', 'প্রতি', 'প্রভৃতি', 'ফিরে', 'বদলে', 'বরং', 'বললেন', 'বললেন না',
    'বললাম', 'বলতে', 'বলল', 'বললেন', 'বলছি', 'বলেছেন', 'বলে', 'বলেন', 'বলা', 'বলে', 'বসে', 'বা', 'বিনা',
    'বিশেষ', 'ব্যবহার', 'ব্যবহার করে', 'ভাল', 'ভাবেই', 'মতো', 'মধ্যেও', 'মধ্যেই', 'মাধ্যমে', 'মোট', 'যখন',
    'যত', 'যথেষ্ট', 'যদি', 'যাবে', 'যাওয়া', 'যাচ্ছে', 'যাতে', 'যার', 'যারা', 'যিনি', 'যে', 'যেকোনো', 'যেখানে',
    'যেমন', 'যেতে', 'যেন', 'যেহেতু', 'যেয়ে', 'র', 'রকম', 'রেখে', 'রয়েছে', 'শুধু', 'সঙ্গে', 'সব', 'সবার',
    'সবাই', 'সমস্ত', 'সম্পর্কে', 'সহ', 'সবারই', 'সামনে', 'সি', 'সে', 'সেই', 'সেখান', 'সেখানে', 'স্পষ্ট',
    'হইতে', 'হইবে', 'হইয়াছে', 'হইয়েছে', 'হওয়া', 'হওয়াতে', 'হওয়ার', 'হওয়ায়', 'হচ্ছে', 'হত', 'হতে',
    'হবেন', 'হয়ে', 'হয়', 'হয়তো', 'হয়নি', 'হয়েছে', 'হয়েছে', 'হল', 'হলে', 'হলো', 'হিসেবে'
]


In [5]:
tokenizer = Tokenizers()
stemmer = BanglaStemmer()

In [6]:

regex_tokenizer = RegexpTokenizer(r'[\u0980-\u09FF]+')
def preprocess_bangla_text(text: str, stem: bool = True) -> list:
    """
    Comprehensive Bangla text preprocessing pipeline
    """
    # 1. Remove non-Bangla characters, URLs, numbers and punctuations
    text = re.sub(r'[^\u0980-\u09FF\s]', '', text)
    
    # 2. Tokenize using regex tokenizer
    tokens = regex_tokenizer.tokenize(text)

    # 3. Stopword removal and aggressive stemming
    processed = []
    for token in tokens:
        token = token.strip()
        if len(token) < 2:
            continue
        if token not in bangla_stopwords:
            if stem:
                stemmed = stemmer.stem(token)
                if stemmed:
                    processed.append(stemmed)
            else:
                processed.append(token)
    return processed


In [7]:
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures

def detect_phrases(docs: list, min_freq: int = 10, 
                   bigram_threshold: float = 7.0, 
                   trigram_threshold: float = 12.0) -> dict:
    """
    Detect meaningful phrases using PMI thresholds.
    Returns: phrase mapping dictionary (e.g., {'রাজশাহীর আম': 'রাজশাহীর_আম'})
    """
    # Flatten the list of tokenized documents into a single token stream
    all_tokens = [token for doc in docs for token in doc]

    # 1. Bigram detection using PMI
    bigram_finder = BigramCollocationFinder.from_words(all_tokens)
    bigram_finder.apply_freq_filter(min_freq)
    bigram_scored = bigram_finder.score_ngrams(BigramAssocMeasures.pmi)
    significant_bigrams = [bigram for bigram, score in bigram_scored if score >= bigram_threshold]

    # 2. Trigram detection using PMI
    trigram_finder = TrigramCollocationFinder.from_words(all_tokens)
    trigram_finder.apply_freq_filter(min_freq)
    trigram_scored = trigram_finder.score_ngrams(TrigramAssocMeasures.pmi)
    significant_trigrams = [trigram for trigram, score in trigram_scored if score >= trigram_threshold]

    # 3. Create a phrase mapping dictionary
    phrase_map = {}
    for bigram in significant_bigrams:
        phrase = ' '.join(bigram)
        merged = '_'.join(bigram)
        phrase_map[phrase] = merged

    for trigram in significant_trigrams:
        phrase = ' '.join(trigram)
        merged = '_'.join(trigram)
        phrase_map[phrase] = merged

    return phrase_map



In [8]:
def integrate_phrases(docs: list, phrase_map: dict) -> list:
    """
    Replace detected phrases in documents with merged tokens
    """
    processed_docs = []
    for doc in docs:
        text = ' '.join(doc)
        # Replace phrases in descending length order (longest first)
        for phrase, merged in sorted(phrase_map.items(), 
                                    key=lambda x: len(x[0]), 
                                    reverse=True):
            if phrase in text:
                text = text.replace(phrase, merged)
        processed_docs.append(text.split())
    return processed_docs

In [9]:
def read_bangla_text_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f if line.strip()]  # remove blank lines

In [13]:
# Example Usage Pipeline
if __name__ == "__main__":
    # Sample Bangla dataset (replace with your actual data)
    # bangla_documents = [
    #     "রাজশাহীর আম বাংলাদেশের সেরা আম হিসেবে পরিচিত",
    #     "চাঁপাইনবাবগঞ্জে আমের বাম্পার ফলন হয়েছে এবার",
    #     "আম রাজশাহী ও চাঁপাইনবাবগঞ্জ অঞ্চলের অর্থনীতির মূল চালিকাশক্তি",
    #     "আম রপ্তানিতে নতুন রেকর্ড গড়ল বাংলাদেশ"
    # ]
    bangla_documents = read_bangla_text_file("pahelabaishak.txt")
    
    # 1. Preprocess all documents
    preprocessed_docs = [preprocess_bangla_text(doc) for doc in bangla_documents]
    
    print("Preprocessed Tokens:")
    for i, doc in enumerate(preprocessed_docs):
        print(f"Doc {i+1}: {doc}")
    
    # 2. Detect significant phrases using PMI
    phrase_mapping = detect_phrases(preprocessed_docs, 
                                  min_freq=5, 
                                  bigram_threshold=4.0,
                                  trigram_threshold=8)
    
    print("\nDetected Phrases:")
    for phrase, merged in phrase_mapping.items():
        print(f"{phrase} => {merged}")
    
    # 3. Integrate phrases into documents
    final_docs = integrate_phrases(preprocessed_docs, phrase_mapping)
    
    print("\nFinal Processed Documents with Phrases:")
    for i, doc in enumerate(final_docs):
        print(f"Doc {i+1}: {doc}")

# Output will show processing stages:
# Preprocessed Tokens: Stemmed tokens without stopwords
# Detected Phrases: Found significant n-grams
# Final Documents: With phrases merged (e.g., ["রাজশাহীর_আম", "বাংলাদেশের", ...])

Preprocessed Tokens:
Doc 1: ['পহেলা', 'বৈশাখ', 'পয়লা', 'বৈশাখ', 'বাংলা', 'পঞ্জিক', 'প্রথম', 'মাস', 'বৈশাখ', 'তারিখ', 'বঙ্গাব্', 'প্রথম', 'দিন', 'তথা', 'বাংলা', 'নববর্ষ', 'দিন', 'সকল', 'বাঙা', 'জাতির', 'ঐতিহ্যবাহী', 'বর্ষবরণ', 'দিন', 'দিন', 'বাংলাদেশ', 'ভারত', 'পশ্চিমবঙ্গ', 'নববর্ষ', 'উৎসব', 'সাথ', 'পালিত', 'ত্রিপুরায়', 'বসবাসরত', 'বাঙালিরা', 'উৎসব', 'অংশ', 'পহেলা', 'বৈশাখ', 'বাংা', 'জাতীয়', 'উৎসব', 'পালিত', 'বাঙালি', 'এক', 'সর্বজনীন', 'লোকউৎসব', 'হিসাব', 'বিবেচিত', 'পহেলা', 'বৈশাখ', 'উদযাপন', 'শুরু', 'হয়', 'পুরান', 'ঢাক', 'মুসলিম', 'মাহিফরাস', 'সম্প্রদায়', 'হা']
Doc 2: ['গ্রেগরীয়', 'বর্ষপঞ্জি', 'অনুসার', 'বাংলাদেশ', 'বছর', '১৪', 'এপ্রিল', 'উৎসব', 'পালিত', 'বাংলা', 'একাডেমি', 'কর্তৃক', 'নির্ধারিত', 'আধুনিক', 'বাংলা', 'পঞ্জিকা', 'অনুসার', 'দিন', 'নির্দিষ্ট', 'পশ্চিমবঙ্গ', 'চান্দ্রসৌর', 'বাংলা', 'পঞ্জিকা', 'অনুসার', '১৫', 'এপ্রিল', 'পহেলা', 'বৈশাখ', 'পালিত', 'এছাড়া', 'দিন', 'বাংলাদেশ', 'পশ্চিমবঙ্গ', 'সরকারি', 'ছু', 'দিন', 'গৃহীত', 'বিভিন্ন', 'পর্যায়', 'ব্যবসায়ী', 'দিন', 'নতুনভাব'

In [14]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel


# 1. Prepare Corpus (using your preprocessed docs)
# -------------------------------------------------
# Assuming you already have:
# final_docs = [['পহেলা', 'বৈশাখ', 'বাংলা'], ['নববর্ষ', 'উৎসব'], ...] 

dictionary = Dictionary(final_docs)
dictionary.filter_extremes(no_below=5, no_above=0.5)  # Adjust thresholds
corpus = [dictionary.doc2bow(doc) for doc in final_docs]

# 2. Train LDA Model
# -------------------
NUM_TOPICS = 4  # Reduced from 5 for more focused topics
lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=NUM_TOPICS,
    random_state=42,
    passes=20,  # Increased iterations
    alpha='asymmetric'  # Favors sparse topic distributions
)

# 3. Evaluate Model Quality
# --------------------------
coherence_model = CoherenceModel(
    model=lda_model,
    texts=final_docs,
    dictionary=dictionary,
    coherence='c_v'
)
print(f"Coherence Score: {coherence_model.get_coherence():.3f}")


# 5. Query Processing Function
# ----------------------------
def get_related_topics_enhanced(user_query, threshold=0.1):
    query_tokens = preprocess_bangla_text(user_query)
    query_bow = dictionary.doc2bow(query_tokens)
    
    # Get document-topic distribution
    topic_dist = lda_model[query_bow]
    
    # Filter topics above threshold
    relevant_topics = [(topic_id, prob) 
                      for topic_id, prob in topic_dist 
                      if prob >= threshold]
    
    # Sort by probability
    relevant_topics.sort(key=lambda x: x[1], reverse=True)
    
    results = []
    for topic_id, prob in relevant_topics:
        top_words = [word for word, _ in lda_model.show_topic(topic_id, topn=7)]
        results.append({
            'topic_id': topic_id,
            'probability': f"{prob:.2%}",
            'top_words': ", ".join(top_words[:5]),
            'all_keywords': top_words
        })
    return results
def display_results(user_input):
    topics = get_related_topics_enhanced(user_input)
    
    print(f"\nQuery: '{user_input}'")
    print(f"Coherence Score: {coherence_model.get_coherence():.3f}")
    print("\nMost Relevant Topics:")
    
    for i, topic in enumerate(topics, 1):
        print(f"{i}. Topic {topic['topic_id']} (Relevance: {topic['probability']})")
        print(f"   Keywords: {topic['top_words']}...")
        if i == 1:  # Show full keywords for top topic
            print(f"   All Keywords: {', '.join(topic['all_keywords'])}")
    print("-"*50)

# 6. Example Usage
# -----------------
display_results("বাংলাদেশ")
display_results("পহেলা বৈশাখ")
display_results("মঙ্গল শোভাযাত্রা")

Coherence Score: 0.498

Query: 'বাংলাদেশ'
Coherence Score: 0.498

Most Relevant Topics:
1. Topic 1 (Relevance: 61.05%)
   Keywords: বছর, বাংলাদেশ, এক, আয়োজন, মেলা...
   All Keywords: বছর, বাংলাদেশ, এক, আয়োজন, মেলা, ঐতিহ্য, এর
2. Topic 0 (Relevance: 21.42%)
   Keywords: উৎসব, দিন, বাংলা, পহেলা_বৈশাখ, এক...
--------------------------------------------------

Query: 'পহেলা বৈশাখ'
Coherence Score: 0.498

Most Relevant Topics:
1. Topic 2 (Relevance: 56.81%)
   Keywords: মাস, দিন, হিন্দু, সাল, ভারত...
   All Keywords: মাস, দিন, হিন্দু, সাল, ভারত, নাম, বাংলা
2. Topic 0 (Relevance: 22.09%)
   Keywords: উৎসব, দিন, বাংলা, পহেলা_বৈশাখ, এক...
3. Topic 1 (Relevance: 13.24%)
   Keywords: বছর, বাংলাদেশ, এক, আয়োজন, মেলা...
--------------------------------------------------

Query: 'মঙ্গল শোভাযাত্রা'
Coherence Score: 0.498

Most Relevant Topics:
1. Topic 0 (Relevance: 38.96%)
   Keywords: উৎসব, দিন, বাংলা, পহেলা_বৈশাখ, এক...
   All Keywords: উৎসব, দিন, বাংলা, পহেলা_বৈশাখ, এক, নববর্ষ, পালিত
2. Topic 

In [15]:
print(f"Label: {topic_labels.get(topic['topic_id'], '')}")

NameError: name 'topic_labels' is not defined